# Dynamic optimization
**ECON2125/6012 Lecture 11**
Fedor Iskhakov

## Announcements & Reminders

- None?
## Plan for this lecture

1. Dynamic decision problems
2. Dynamic discrete choice
3. Cake eating and consumption/savings decisions

```{image} _static/img/coming_soon.png
   :scale: 100 %
   :alt: soming soon
   :align: center
```